In [1]:
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
load_dotenv()

True

In [7]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.checkpoint.redis import RedisSaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
# llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model= 'gemma-3-1b-it-qat',
    api_key="api_key",  # API key có thể là bất kỳ giá trị nào
    base_url="http://localhost:1234/v1"  # Địa chỉ server LM Studio
)
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]

llm_with_tools = llm.bind_tools(tools)
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition, # tool_args khác rỗng thì chạy tiếp
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [8]:
# -----------------------------------
# 1. Chạy chatbot với input từ user
# -----------------------------------

while True:
    user_input = input("User: ")
    if user_input.lower() in {"exit", "quit"}:
        break

    # Gửi tin nhắn vào graph
    events = graph.stream(
        {"messages": [("user", user_input)]},
        config={"configurable": {"thread_id": "1"}},
        stream_mode="values"
    )


    # -----------------------------------
    # 2. Lấy câu trả lời cuối từ chatbot
    # -----------------------------------
    for state in events:
        # State cuối cùng chứa tất cả message
        last_message = state["messages"][-1]  # lấy message cuối
        if hasattr(last_message, "content"):  # kiểu BaseMessage
            print("Bot:", last_message.content)
        else:
            print("Bot:", last_message)  # nếu chỉ là str
        print(state)


Bot: xin chào
{'messages': [HumanMessage(content='xin chào', additional_kwargs={}, response_metadata={}, id='b0621324-6a00-46e9-94db-e411f54083a6')]}
08:55:44 httpx INFO   HTTP Request: POST http://localhost:1234/v1/chat/completions "HTTP/1.1 200 OK"
Bot: 
{'messages': [HumanMessage(content='xin chào', additional_kwargs={}, response_metadata={}, id='b0621324-6a00-46e9-94db-e411f54083a6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '955520787', 'function': {'arguments': '{"ticker":"TSLA"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 2441, 'total_tokens': 2510, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-1b-it-qat', 'system_fingerprint': 'gemma-3-1b-it-qat', 'id': 'chatcmpl-pscoi890ppykpnvpasjw', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e596c330-bfcc-4021-90a8-06af2339576e-0', tool_c

In [ ]:
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage


# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["poem", "story", "joke"] = Field(
        None, description="The next step in the routing process"
    )


# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


# State
class State(TypedDict):
    input: str
    decision: str
    output: str


# Nodes
def llm_call_1(state: State):
    """Write a story"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_2(state: State):
    """Write a joke"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_3(state: State):
    """Write a poem"""

    result = llm.invoke(state["input"])
    return {"output": result.content}


def llm_call_router(state: State):
    """Route the input to the appropriate node"""

    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            SystemMessage(
                content="Route the input to story, joke, or poem based on the user's request."
            ),
            HumanMessage(content=state["input"]),
        ]
    )

    return {"decision": decision.step}


# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    # Return the node name you want to visit next
    if state["decision"] == "story":
        return "llm_call_1"
    elif state["decision"] == "joke":
        return "llm_call_2"
    elif state["decision"] == "poem":
        return "llm_call_3"


# Build workflow
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)

# Add edges to connect nodes
router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {  # Name returned by route_decision : Name of next node to visit
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    },
)
router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)

# Compile workflow
router_workflow = router_builder.compile()

# Show the workflow
display(Image(router_workflow.get_graph().draw_mermaid_png()))

# Invoke
state = router_workflow.invoke({"input": "Write me a joke about cats"})
print(state["output"])

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model= '',
    api_key="api_key",  # API key có thể là bất kỳ giá trị nào
    base_url="http://192.168.4.143:1234/v1"  # Địa chỉ server LM Studio
)

In [20]:
while True:
    message = input()
    if message == 'quit':
        break
    response = llm.invoke(message)
    print(response.content)

11:44:24 httpx INFO   HTTP Request: POST http://192.168.4.143:1234/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': 'Model unloaded.'}

In [9]:
from mistralai import Mistral
from dotenv import load_dotenv
import datauri
import os

In [10]:
load_dotenv()
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)